In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
train = pd.read_csv('Train_data.csv', index_col=False)
o_test = pd.read_csv('Test_data.csv', index_col=False)

In [3]:
occurences = train['accident'].value_counts()
print(occurences)
#print(train)

0    59446
1      880
Name: accident, dtype: int64


In [4]:
train.head()

,Unnamed: 0,code,clientType,registrationMode,planName,accident,duration,country,netSales,netProfit,gender,age
0,1,CBH,Basic,Offline,M,0,186,MALAYSIA,-29.0,9.57,F,81
1,2,CBH,Basic,Offline,M,0,186,MALAYSIA,-29.0,9.57,F,71
2,3,CWT,Basic,Online,Q,0,65,AUSTRALIA,-49.5,29.70,NaN,32
3,4,CWT,Basic,Online,Q,0,60,AUSTRALIA,-39.6,23.76,NaN,32
4,5,CWT,Basic,Online,Q,0,79,ITALY,-19.8,11.88,NaN,41


In [5]:
o_test.head()

,Unnamed: 0,code,clientType,registrationMode,planName,duration,country,netSales,netProfit,gender,age
0,42104,EPX,Basic,Online,B,24,INDIA,26.0,0.00,NaN,36
1,869,EPX,Basic,Online,K,52,SINGAPORE,11.0,0.00,NaN,36
2,57025,CWT,Basic,Online,Q,6,MALAYSIA,39.6,23.76,NaN,45
3,1148,EPX,Basic,Online,K,37,MALAYSIA,10.0,0.00,NaN,25
4,23356,CWT,Basic,Online,Q,77,JAPAN,29.7,17.82,NaN,59


In [6]:
print(train.shape, o_test.shape)
print(train.dtypes)
print(o_test.dtypes)

(60326, 12) (3000, 11)
Unnamed: 0            int64
code                 object
clientType           object
registrationMode     object
planName             object
accident              int64
duration              int64
country              object
netSales            float64
netProfit           float64
gender               object
age                   int64
dtype: object
Unnamed: 0            int64
code                 object
clientType           object
registrationMode     object
planName             object
duration              int64
country              object
netSales            float64
netProfit           float64
gender               object
age                   int64
dtype: object


In [7]:
def str_to_int(df):
    str_columns = df.select_dtypes(['object']).columns
    print(str_columns)
    for col in str_columns:
        df[col] = df[col].astype('category')

    cat_columns = df.select_dtypes(['category']).columns
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    return df

In [8]:
str_to_int(train)
str_to_int(o_test)

Index(['code', 'clientType', 'registrationMode', 'planName', 'country',
       'gender'],
      dtype='object')
Index(['code', 'clientType', 'registrationMode', 'planName', 'country',
       'gender'],
      dtype='object')


,Unnamed: 0,code,clientType,registrationMode,planName,duration,country,netSales,netProfit,gender,age
0,42104,7,0,1,1,24,29,26.00,0.00,-1,36
1,869,7,0,1,10,52,61,11.00,0.00,-1,36
2,57025,6,0,1,15,6,42,39.60,23.76,-1,45
3,1148,7,0,1,10,37,42,10.00,0.00,-1,25
4,23356,6,0,1,15,77,34,29.70,17.82,-1,59
...,...,...,...,...,...,...,...,...,...,...,...
2995,58310,7,0,1,10,3,11,19.00,0.00,-1,35
2996,5892,7,0,1,10,10,1,54.00,0.00,-1,36
2997,23746,13,1,1,21,16,61,1.22,0.34,-1,48
2998,3667,7,0,1,1,7,54,33.00,0.00,-1,39


In [9]:
independent = list(train.columns.values)
independent.remove('accident')
print(independent)
X = train[independent]
y = train['accident']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


['Unnamed: 0', 'code', 'clientType', 'registrationMode', 'planName', 'duration', 'country', 'netSales', 'netProfit', 'gender', 'age']


In [10]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(11,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=1)
test_loss, test_acc = model.evaluate(X_test, y_test)

Train on 42228 samples
Epoch 1/3
42228/42228 [==============================] - 48s 1ms/sample - loss: 5.0081 - accuracy: 0.9687s - loss: 5.0269 - ac
Epoch 2/3
42228/42228 [==============================] - 49s 1ms/sample - loss: 0.1534 - accuracy: 0.9842
Epoch 3/3
18098/18098 [==============================] - 0s 26us/sample - loss: 0.0757 - accuracy: 0.9855


In [11]:
print('Accuracy: ', test_acc)

Accuracy:  0.98546803


In [15]:
o = o_test
pred = model.predict(o)
print(pred.shape)
output = np.empty([3000,1])
file = open("output.txt", "w")
file2 = open("output_prob.txt", "w")
#print(output)
print(len(o))
for i in range(len(pred)):
    file2.write("" + str(pred[i]))
    file2.write("\n")
    #print(pred[i])
    if pred[i] > 0.5:
        #print(1)
        file.write("1\n")
    else:
        #print(0)
        file.write("0\n")
        
file.close()
file2.close()

#print(output)
#np.savetxt('file2.txt', output, delimiter='\n')

(3000, 1)
3000
